# 39차시: [프로젝트] 투자 분석 자동화 시스템 구축 - 실전 템플릿
====================================================

학습 목표:
- Module 01-04의 모든 학습 내용을 종합
- '나만의 투자 분석 자동화 시스템' 구축
- 데이터 수집 → 분석 → AI 추천 → 리포팅 파이프라인 완성

프로젝트 범위:
- 개인화된 투자 분석 자동화 시스템
- AI 기반 최적 종목 추천 시스템 (LangChain 활용)
- 원시 데이터를 AI에게 직접 전달하여 전문가 수준의 종합 판단 수행
- 일일/주간 리포트 자동 생성
- (선택) 이메일 발송

주요 구현 내용:  
여러 종목의 최근 주가 데이터를 수집한 뒤,  
- 기술적 분석(이동평균, RSI, MACD로 추세·과매수/과매도·매수/매도 신호 판단),  
- 통계적 분석(기간수익률, 변동성, 샤프비율로 수익 대비 위험 평가)  
를 각 종목별로 수행합니다. 그 후 이 두 가지 분석 결과의 원시 데이터를  
LangChain을 활용하여 AI에게 직접 전달하고, AI가 전문가처럼 종합적으로 분석하여
1) 최적의 투자 종목 1-3개를 추천하고,
2) 각 추천 종목에 대한 구체적인 추천 이유를 제공하며,
3) PDF·Excel 리포트와 차트로 자동 생성하고, 이메일로 발송합니다.

AI 모델 선택:
- OpenAI (gpt-5-mini) 또는 Google Gemini (gemini-2.5-flash) 중 선택 가능
- LangChain을 통해 통일된 인터페이스로 모델 전환 가능
- 점수화나 랭킹 없이 원시 데이터를 직접 분석하여 더 객관적이고 전문적인 추천 제공  

In [1]:
# !pip install -Uq finance-datareader pandas-datareader reportlab openpyxl python-dotenv openai beautifulsoup4 requests langchain

In [2]:
import os
import re
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 데이터 수집
import FinanceDataReader as fdr
import pandas_datareader.data as web
import requests
from bs4 import BeautifulSoup

# 리포트
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image as RLImage
from reportlab.lib.units import cm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# Excel
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image as XLImage

# 이메일
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# 랭체인
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

# 환경 변수
from dotenv import load_dotenv

# 한글 폰트
try:
    import koreanize_matplotlib
except:
    plt.rcParams['font.family'] = 'Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] = False

load_dotenv()

True

## 1. 폰트 설정

한글 폰트를 등록하여 PDF와 차트에서 한글이 정상적으로 표시되도록 합니다.
**참고: Module 04 - 37차시 (PDF/Excel 리포트 자동화)**

In [4]:
# ============================================
# 1. 한글 폰트 등록
# **참고: Module 04 - 37차시 (PDF/Excel 리포트 자동화)**
# ============================================
font_paths = [
    'C:/Windows/Fonts/malgun.ttf',  # Windows
    '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',  # Linux/Colab
    '/System/Library/Fonts/AppleGothic.ttf'  # Mac
]

font_registered = False
for path in font_paths:
    if os.path.exists(path):
        try:
            pdfmetrics.registerFont(TTFont('Korean', path))
            print(f"[폰트 등록 성공]: {path}")
            font_registered = True
            break
        except Exception as e:
            print(f"[폰트 등록 실패]: {e}")
            continue

if not font_registered:
    print("[경고] 한글 폰트를 찾을 수 없습니다. 영문만 사용됩니다.")

[폰트 등록 성공]: C:/Windows/Fonts/malgun.ttf


## 2. 데이터 수집 모듈

주가 데이터, 경제 지표, 뉴스 데이터를 수집하는 함수들입니다.
**참고:**
- Module 01 - 8차시 (국내 주식 데이터 수집 및 기본 분석): FinanceDataReader 사용
- Module 02 - 13차시 (FRED API로 글로벌 경제 지표 수집): pandas_datareader로 FRED 데이터 수집
- Module 02 - 15차시 (네이버 금융에서 뉴스 타이틀과 시장 지표 크롤링): BeautifulSoup, requests를 이용한 뉴스 크롤링

In [5]:
# ============================================
# 2. 데이터 수집 모듈
# **참고:**
#   - Module 01 - 8차시: FinanceDataReader로 주가 데이터 수집
#   - Module 02 - 13차시: FRED API로 경제 지표 수집
#   - Module 02 - 15차시: 네이버 금융 뉴스 크롤링
# ============================================

# 종목명 딕셔너리 (전역 변수)
STOCK_NAMES = {
    "005930": "삼성전자",
    "000660": "SK하이닉스",
    "035420": "NAVER",
    "051910": "LG화학",
    "006400": "삼성SDI",
    "035720": "카카오",
    "005380": "현대차",
    "005490": "POSCO홀딩스",
    "028260": "삼성물산",
    "105560": "KB금융",
    "000270": "기아",
    "034730": "SK",
    "006570": "대한항공",
    "003550": "LG",
    "051900": "LG생활건강",
    # 필요한 종목 추가 가능
}

def collect_stock_data(stock_code: str, start_date, end_date) -> pd.DataFrame:
    """
    주가 데이터 수집 (FinanceDataReader)
    
    Parameters:
        stock_code: 종목코드 (예: "005930")
        start_date: 시작일 (date, datetime 또는 'YYYY-MM-DD' 문자열)
        end_date: 종료일
    
    Returns:
        pd.DataFrame: OHLCV 데이터
    """
    try:
        # FDR은 datetime 객체 또는 'YYYY-MM-DD' 형식 사용
        # date 객체는 그대로 사용 가능
        df = fdr.DataReader(stock_code, start_date, end_date)
        return df
    except Exception as e:
        print(f"[에러] 주가 데이터 수집 실패 ({stock_code}): {e}")
        return pd.DataFrame()

def collect_multiple_stocks(stock_codes: list, start_date, end_date) -> dict:
    """
    여러 종목 데이터 일괄 수집
    
    Parameters:
        stock_codes: 종목코드 리스트
        start_date: 시작일
        end_date: 종료일
    
    Returns:
        dict: {종목코드: DataFrame} 형태
    """
    result = {}
    for code in stock_codes:
        df = collect_stock_data(code, start_date, end_date)
        if not df.empty:
            result[code] = df
    return result

def collect_economic_data(series_id: str, start_date, end_date) -> pd.DataFrame:
    """
    경제 지표 수집 (FRED API)
    
    Parameters:
        series_id: FRED 시리즈 ID (예: "FEDFUNDS")
        start_date: 시작일
        end_date: 종료일
    
    Returns:
        pd.DataFrame: 경제 지표 데이터
    """
    try:
        df = web.DataReader(series_id, 'fred', start_date, end_date)
        return df
    except Exception as e:
        print(f"[에러] 경제 지표 수집 실패 ({series_id}): {e}")
        return pd.DataFrame()

def collect_multiple_economic_indicators(series_ids: list, start_date, end_date) -> pd.DataFrame:
    """
    여러 경제 지표 수집 및 병합
    
    Parameters:
        series_ids: FRED 시리즈 ID 리스트
        start_date: 시작일
        end_date: 종료일
    
    Returns:
        pd.DataFrame: 병합된 경제 지표 데이터
    """
    dfs = []
    for sid in series_ids:
        df = collect_economic_data(sid, start_date, end_date)
        if not df.empty:
            dfs.append(df)
    
    if dfs:
        return pd.concat(dfs, axis=1)
    return pd.DataFrame()

def crawl_news(stock_code: str, max_pages: int = 3) -> list:
    """
    네이버 금융 뉴스 크롤링 (간단 버전)
    
    Parameters:
        stock_code: 종목코드
        max_pages: 최대 페이지 수
    
    Returns:
        list: 뉴스 제목 리스트
    """
    news_list = []
    try:
        for page in range(1, max_pages + 1):
            url = f"https://finance.naver.com/item/news.naver?code={stock_code}&page={page}"
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers, timeout=5)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                titles = soup.find_all('a', class_='title')
                for title in titles[:5]:  # 페이지당 최대 5개
                    news_list.append(title.get_text(strip=True))
    except Exception as e:
        print(f"[경고] 뉴스 크롤링 실패: {e}")
    
    return news_list[:10]  # 최대 10개 반환

def get_stock_name(stock_code: str) -> str:
    """종목명 조회 (딕셔너리 사용)"""
    return STOCK_NAMES.get(stock_code, stock_code)  # 딕셔너리에 없으면 종목코드 그대로 사용

## 3. 분석 모듈

기술적 분석, 통계 분석, AI 분석을 수행하는 함수들입니다.
**참고:**
- Module 01 - 9차시 (이동평균을 이용한 금융 시계열 추세 분석): 이동평균, RSI, MACD 계산
- Module 01 - 10차시 (포트폴리오와 최적화): 통계 지표 계산 (수익률, 샤프비율 등)
- Module 03 - 27차시 (생성형AI LLM 투자리서치): OpenAI API를 이용한 AI 분석
- Module 03 - 30차시 (LangChain 투자분석봇): LangChain을 이용한 모델 선택 (OpenAI/Gemini)

In [6]:
# ============================================
# 3. 분석 모듈
# **참고:**
#   - Module 01 - 9차시: 이동평균, RSI, MACD 계산 및 기술적 분석
#   - Module 01 - 10차시: 통계 지표 계산 (수익률, 샤프비율, 변동성 등)
#   - Module 03 - 27차시: OpenAI API를 이용한 AI 분석
#   - Module 03 - 30차시: LangChain을 이용한 모델 선택 (OpenAI/Gemini)
# ============================================

def calculate_returns(df: pd.DataFrame, price_col: str = 'Close') -> pd.DataFrame:
        """
        수익률 계산
        
        Parameters:
            df: 주가 데이터
            price_col: 가격 컬럼명
        
        Returns:
            pd.DataFrame: 수익률이 추가된 데이터
        """
        df = df.copy()
        df['일간수익률'] = df[price_col].pct_change() * 100
        df['누적수익률'] = ((1 + df['일간수익률'] / 100).cumprod() - 1) * 100
        return df
    
def add_moving_averages(df: pd.DataFrame, price_col: str = 'Close', 
                        periods: list = [5, 20, 60]) -> pd.DataFrame:
        """
        이동평균선 추가
        
        Parameters:
            df: 주가 데이터
            price_col: 가격 컬럼명
            periods: 이동평균 기간 리스트
        
        Returns:
            pd.DataFrame: 이동평균이 추가된 데이터
        """
        df = df.copy()
        for period in periods:
            df[f'MA{period}'] = df[price_col].rolling(window=period).mean()
        return df
    
def calculate_rsi(df: pd.DataFrame, price_col: str = 'Close', period: int = 14) -> pd.DataFrame:
        """
        RSI (Relative Strength Index) 계산
        
        Parameters:
            df: 주가 데이터
            price_col: 가격 컬럼명
            period: RSI 기간 (기본 14일)
        
        Returns:
            pd.DataFrame: RSI가 추가된 데이터
        """
        df = df.copy()
        delta = df[price_col].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        return df
    
def calculate_macd(df: pd.DataFrame, price_col: str = 'Close', 
                  fast: int = 12, slow: int = 26, signal: int = 9) -> pd.DataFrame:
        """
        MACD (Moving Average Convergence Divergence) 계산
        
        Parameters:
            df: 주가 데이터
            price_col: 가격 컬럼명
            fast: 빠른 이동평균 기간
            slow: 느린 이동평균 기간
            signal: 신호선 기간
        
        Returns:
            pd.DataFrame: MACD가 추가된 데이터
        """
        df = df.copy()
        ema_fast = df[price_col].ewm(span=fast, adjust=False).mean()
        ema_slow = df[price_col].ewm(span=slow, adjust=False).mean()
        
        df['MACD'] = ema_fast - ema_slow
        df['MACD_Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
        
        return df
    
def calculate_statistics(df: pd.DataFrame, price_col: str = 'Close') -> dict:
        """
        기본 통계 계산
        
        Parameters:
            df: 주가 데이터
            price_col: 가격 컬럼명
        
        Returns:
            dict: 통계 정보
        """
        returns = df[price_col].pct_change().dropna()
        
        stats = {
            '시작가': df[price_col].iloc[0],
            '종료가': df[price_col].iloc[-1],
            '최고가': df[price_col].max() if 'High' not in df.columns else df['High'].max(),
            '최저가': df[price_col].min() if 'Low' not in df.columns else df['Low'].min(),
            '평균': df[price_col].mean(),
            '표준편차': df[price_col].std(),
            '기간수익률': ((df[price_col].iloc[-1] / df[price_col].iloc[0]) - 1) * 100,
            '일평균수익률': returns.mean() * 100,
            '변동성': returns.std() * 100,
        }
        
        # 샤프비율 계산 (무위험 수익률 0% 가정)
        if len(returns) > 0 and returns.std() > 0:
            stats['샤프비율'] = (returns.mean() / returns.std()) * np.sqrt(252)  # 연율화
        else:
            stats['샤프비율'] = 0
        
        return stats
    
def technical_analysis(df: pd.DataFrame) -> dict:
        """
        기술적 분석 종합
        
        Parameters:
            df: 주가 데이터 (이미 이동평균, RSI, MACD가 계산된 상태)
        
        Returns:
            dict: 기술적 분석 결과
        """
        result = {}
        
        # 이동평균 교차 확인
        if 'MA5' in df.columns and 'MA20' in df.columns:
            latest_ma5 = df['MA5'].iloc[-1]
            latest_ma20 = df['MA20'].iloc[-1]
            prev_ma5 = df['MA5'].iloc[-2] if len(df) > 1 else latest_ma5
            prev_ma20 = df['MA20'].iloc[-2] if len(df) > 1 else latest_ma20
            
            # 골든크로스/데드크로스
            if latest_ma5 > latest_ma20 and prev_ma5 <= prev_ma20:
                result['이동평균_신호'] = '골든크로스'
            elif latest_ma5 < latest_ma20 and prev_ma5 >= prev_ma20:
                result['이동평균_신호'] = '데드크로스'
            else:
                result['이동평균_신호'] = '중립'
        
        # RSI 신호
        if 'RSI' in df.columns:
            latest_rsi = df['RSI'].iloc[-1]
            if latest_rsi > 70:
                result['RSI_신호'] = '과매수'
            elif latest_rsi < 30:
                result['RSI_신호'] = '과매도'
            else:
                result['RSI_신호'] = '중립'
            result['RSI_값'] = latest_rsi
        
        # MACD 신호
        if 'MACD' in df.columns and 'MACD_Signal' in df.columns:
            latest_macd = df['MACD'].iloc[-1]
            latest_signal = df['MACD_Signal'].iloc[-1]
            if latest_macd > latest_signal:
                result['MACD_신호'] = '매수'
            else:
                result['MACD_신호'] = '매도'
            result['MACD_값'] = latest_macd
        
        return result

## 4. AI 최적 종목 추천 모듈

전체 종목의 원시 분석 데이터를 AI에게 전달하여 최적 종목을 추천받는 함수입니다.  
**참고: Module 03 - 30차시 (LangChain 투자분석봇): LangChain을 이용한 모델 선택 (OpenAI/Gemini)**

In [7]:
def recommend_optimal_stocks(stock_analyses: dict, model_provider: str = "openai") -> dict:
    """
    전체 종목의 원시 분석 데이터를 바탕으로 최적 종목 추천 (LangChain 사용)
    
    Parameters:
        stock_analyses: {종목코드: {기술적분석, 통계분석}} 형태 (원시 데이터)
        model_provider: 모델 제공자 ("openai" 또는 "google_genai")
    
    Returns:
        dict: 추천 결과 (추천종목, 추천이유)
    """
    
    # 모델 선택
    if model_provider == "openai":
        model_name = "gpt-5-mini"
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            return {
                '추천종목': [],
                '추천이유': 'OpenAI API 키가 설정되지 않았습니다.'
            }
    elif model_provider == "google_genai":
        model_name = "gemini-2.5-flash"
        api_key = os.getenv('GOOGLE_API_KEY')
        if not api_key:
            return {
                '추천종목': [],
                '추천이유': 'Google API 키가 설정되지 않았습니다.'
            }
    else:
        return {
            '추천종목': [],
            '추천이유': f'지원하지 않는 모델 제공자입니다: {model_provider}'
        }
    
    try:
        # LangChain 모델 초기화
        model = init_chat_model(model_name, model_provider=model_provider)
        
        # 전체 종목 원시 데이터 준비
        stocks_data = []
        for stock_code, analysis in stock_analyses.items():
            tech_result = analysis.get('technical', {})
            stats = analysis.get('statistical', {})
            
            stocks_data.append({
                '종목명': analysis.get('stock_name', stock_code),
                '종목코드': stock_code,
                '기술적분석': tech_result,
                '통계지표': stats
            })
        
        # 프롬프트 구성 (원시 데이터 전달)
        stocks_text = "\n".join([
            f"{i+1}. {s['종목명']} ({s['종목코드']}):\n"
            f"   - 기술적 분석: "
            f"이동평균신호={s['기술적분석'].get('이동평균_신호', 'N/A')}, "
            f"RSI신호={s['기술적분석'].get('RSI_신호', 'N/A')} "
            f"(RSI값={s['기술적분석'].get('RSI_값', 0):.1f}), "
            f"MACD신호={s['기술적분석'].get('MACD_신호', 'N/A')}\n"
            f"   - 통계 지표: "
            f"기간수익률={s['통계지표'].get('기간수익률', 0):.2f}%, "
            f"샤프비율={s['통계지표'].get('샤프비율', 0):.2f}, "
            f"변동성={s['통계지표'].get('변동성', 0):.2f}%, "
            f"일평균수익률={s['통계지표'].get('일평균수익률', 0):.2f}%"
            for i, s in enumerate(stocks_data)
        ])
        
        system_prompt = """당신은 15년 경력의 증권 애널리스트입니다.
여러 종목의 기술적 분석 결과와 통계 지표를 종합적으로 검토하여 최적의 투자 종목을 추천하고, 
객관적이고 실용적인 추천 이유를 제공합니다.
한국어로 답변하며, 구체적이고 실용적인 조언을 제공합니다.
점수나 랭킹에 의존하지 말고, 원시 데이터를 직접 분석하여 판단하세요."""
        
        user_prompt = f"""다음은 여러 종목의 기술적 분석 결과와 통계 지표입니다:

[종목별 분석 데이터]
{stocks_text}

위 종목들의 원시 데이터를 직접 분석하여:
1. 최적의 투자 종목 1-3개를 추천해주세요
2. 각 추천 종목에 대한 구체적인 추천 이유를 설명해주세요
   - 기술적 분석 신호(이동평균, RSI, MACD)의 의미
   - 통계 지표(수익률, 샤프비율, 변동성)의 해석
   - 위험 대비 수익의 종합 평가

다음 형식으로 답변해주세요:

## 추천 종목:
1. [종목명] ([종목코드])
2. [종목명] ([종목코드])
3. [종목명] ([종목코드]) (선택)

## 추천 이유:
[각 종목별로 구체적인 추천 이유를 설명]
"""
        
        # LangChain으로 모델 호출
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        
        response = model.invoke(messages)
        
        # 응답 텍스트 추출
        if hasattr(response, 'content'):
            analysis_text = response.content
        else:
            analysis_text = str(response)
        
        # 추천 종목 추출
        recommended_stocks = []
        if "추천 종목:" in analysis_text or "추천종목:" in analysis_text:
            lines = analysis_text.split('\n')
            in_recommendation = False
            for line in lines:
                if '추천 종목:' in line or '추천종목:' in line:
                    in_recommendation = True
                    continue
                if in_recommendation and line.strip():
                    if line.strip().startswith('##'):
                        break
                    # 종목코드 추출 (괄호 안의 6자리 숫자)
                    codes = re.findall(r'\((\d{6})\)', line)
                    if codes:
                        recommended_stocks.append(codes[0])
                    if len(recommended_stocks) >= 3:
                        break
        
        # 추천 이유 추출
        recommendation_reason = ""
        if "추천 이유:" in analysis_text or "추천이유:" in analysis_text:
            reason_lines = []
            in_reason = False
            for line in analysis_text.split('\n'):
                if '추천 이유:' in line or '추천이유:' in line:
                    in_reason = True
                    reason_lines.append(line.split(':')[-1].strip())
                    continue
                if in_reason and line.strip():
                    if line.strip().startswith('##'):
                        break
                    reason_lines.append(line.strip())
            recommendation_reason = '\n'.join(reason_lines)
        
        # 추천 종목이 없으면 첫 번째 종목을 기본값으로
        if not recommended_stocks and stock_analyses:
            recommended_stocks = [list(stock_analyses.keys())[0]]
        
        return {
            '추천종목': recommended_stocks,
            '추천이유': recommendation_reason or analysis_text[:500]  # 전체 응답의 처음 500자
        }
        
    except Exception as e:
        print(f"[경고] AI 추천 실패: {e}")
        # 기본값: 첫 번째 종목
        default_stock = list(stock_analyses.keys())[0] if stock_analyses else None
        return {
            '추천종목': [default_stock] if default_stock else [],
            '추천이유': f'AI 추천 중 오류 발생: {str(e)}'
        }

## 5. 시각화 모듈

차트를 생성하는 함수들입니다.
**참고:**
- Module 01 - 5차시 (데이터 시각화, Matplotlib과 Seaborn 기초): matplotlib 기본 사용법
- Module 01 - 7차시 (금융 시계열 데이터 시각화): 주가 차트 그리기
- Module 04 - 37차시 (PDF/Excel 리포트 자동화): 차트 이미지 저장

In [8]:
# ============================================
# 5. 시각화 모듈
# **참고:**
#   - Module 01 - 5차시: matplotlib 기본 사용법
#   - Module 01 - 7차시: 주가 차트 그리기
#   - Module 04 - 37차시: 차트 이미지 저장
# ============================================

def create_price_chart(df: pd.DataFrame, title: str, 
                           price_col: str = 'Close',
                           ma_cols: list = None,
                           output_path: str = None) -> str:
        """
        주가 차트 생성
        
        Parameters:
            df: 주가 데이터
            title: 차트 제목
            price_col: 가격 컬럼
            ma_cols: 이동평균 컬럼 리스트
            output_path: 저장 경로
        
        Returns:
            str: 저장된 파일 경로
        """
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # 종가
        ax.plot(df.index, df[price_col], label=price_col, linewidth=2, color='blue')
        
        # 이동평균선
        if ma_cols:
            colors_list = ['orange', 'green', 'red', 'purple']
            for i, col in enumerate(ma_cols):
                if col in df.columns:
                    ax.plot(df.index, df[col], label=col, 
                            linewidth=1, color=colors_list[i % len(colors_list)])
        
        ax.set_title(title, fontsize=14, fontweight='bold')
        ax.set_xlabel('날짜')
        ax.set_ylabel('가격')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if output_path:
            plt.savefig(output_path, dpi=150, bbox_inches='tight')
            plt.close()
            return output_path
        else:
            plt.show()
            return None
    
def create_recommendation_chart(ai_recommendation: dict, stock_analyses: dict, output_path: str = None) -> str:
        """
        AI 추천 종목 차트 생성
        
        Parameters:
            ai_recommendation: AI 추천 결과 (추천종목, 추천이유)
            stock_analyses: 종목 분석 데이터
            output_path: 저장 경로
        
        Returns:
            str: 저장된 파일 경로
        """
        fig, ax = plt.subplots(figsize=(10, 6))
        
        recommended_stocks = ai_recommendation.get('추천종목', [])
        if not recommended_stocks:
            plt.close()
            return None
        
        # 추천 종목의 통계 지표 표시
        stock_names = []
        total_returns = []
        
        for code in recommended_stocks[:5]:  # 최대 5개
            if code in stock_analyses:
                analysis = stock_analyses[code]
                stock_name = analysis.get('stock_name', code)
                stats = analysis.get('statistical', {})
                total_return = stats.get('기간수익률', 0)
                
                stock_names.append(f"{stock_name}\n({code})")
                total_returns.append(total_return)
        
        if not stock_names:
            plt.close()
            return None
        
        x_pos = np.arange(len(stock_names))
        colors_list = ['steelblue', 'green', 'orange', 'red', 'purple']
        bars = ax.barh(x_pos, total_returns, color=colors_list[:len(stock_names)], alpha=0.7)
        
        ax.set_yticks(x_pos)
        ax.set_yticklabels(stock_names)
        ax.set_xlabel('기간수익률 (%)')
        ax.set_title('AI 추천 종목 (기간수익률)', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
        
        # 수익률 표시
        for i, (name, ret) in enumerate(zip(stock_names, total_returns)):
            ax.text(ret + 0.5 if ret >= 0 else ret - 0.5, i, f"{ret:.1f}%", 
                   va='center', fontsize=9, ha='left' if ret >= 0 else 'right')
        
        plt.tight_layout()
        
        if output_path:
            plt.savefig(output_path, dpi=150, bbox_inches='tight')
            plt.close()
            return output_path
        else:
            plt.show()
            return None

## 6. 리포트 모듈

PDF/Excel 리포트 생성 및 이메일 발송 기능을 제공하는 함수들입니다.
**참고:**
- Module 04 - 37차시 (분석리포트 자동화 PDF/Excel): reportlab, openpyxl을 이용한 리포트 생성
- Module 02 - 17차시 (자동화 스케줄링): 이메일 발송 기능

In [9]:
# ============================================
# 6. 리포트 모듈
# **참고:**
#   - Module 04 - 37차시: PDF/Excel 리포트 생성 (reportlab, openpyxl)
#   - Module 02 - 17차시: 이메일 발송 기능
# ============================================

def generate_pdf_report(ai_recommendation: dict, stock_analyses: dict, chart_path: str = None,
                       output_path: str = None) -> str:
        """
        PDF 리포트 생성
        
        Parameters:
            ai_recommendation: AI 추천 결과
            stock_analyses: 종목 분석 데이터
            chart_path: 차트 이미지 경로 (선택)
            output_path: 출력 파일 경로
        
        Returns:
            str: 생성된 파일 경로
        """
        if output_path is None:
            output_dir = "output"
            os.makedirs(output_dir, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = os.path.join(output_dir, f"stock_recommendation_report_{timestamp}.pdf")
        
        doc = SimpleDocTemplate(output_path, pagesize=A4)
        styles = getSampleStyleSheet()
        
        # 한글 스타일 추가
        if font_registered:
            if 'Korean' not in [s.name for s in styles.byName.values()]:
                styles.add(ParagraphStyle(
                    name='Korean',
                    fontName='Korean',
                    fontSize=12,
                    leading=16
                ))
                styles.add(ParagraphStyle(
                    name='KoreanTitle',
                    fontName='Korean',
                    fontSize=18,
                    leading=22,
                    spaceAfter=20
                ))
        
        story = []
        
        # 제목
        title = "AI 투자 종목 추천 리포트"
        if font_registered:
            story.append(Paragraph(title, styles['KoreanTitle']))
        else:
            story.append(Paragraph(title, styles['Title']))
        story.append(Spacer(1, 20))
        
        # 생성 시각
        timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        if font_registered:
            story.append(Paragraph(f"생성 시각: {timestamp_str}", styles['Korean']))
        else:
            story.append(Paragraph(f"Generated: {timestamp_str}", styles['Normal']))
        story.append(Spacer(1, 20))
        
        # AI 추천 종목 섹션
        recommended_stocks = ai_recommendation.get('추천종목', [])
        if recommended_stocks:
            if font_registered:
                story.append(Paragraph("AI 추천 종목", styles['KoreanTitle']))
            else:
                story.append(Paragraph("AI Recommended Stocks", styles['Title']))
            story.append(Spacer(1, 10))
            
            # 추천 종목 테이블
            table_data = [['순위', '종목명', '종목코드', '기간수익률', '샤프비율', '변동성']]
            
            for i, code in enumerate(recommended_stocks, 1):
                if code in stock_analyses:
                    analysis = stock_analyses[code]
                    stock_name = analysis.get('stock_name', code)
                    stats = analysis.get('statistical', {})
                    table_data.append([
                        str(i),
                        stock_name,
                        code,
                        f"{stats.get('기간수익률', 0):.2f}%",
                        f"{stats.get('샤프비율', 0):.2f}",
                        f"{stats.get('변동성', 0):.2f}%"
                    ])
            
            if len(table_data) > 1:  # 헤더 외에 데이터가 있으면
                table = Table(table_data)
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black),
                    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                    ('FONTSIZE', (0, 0), (-1, -1), 9)
                ]))
                
                if font_registered:
                    table.setStyle(TableStyle([('FONTNAME', (0, 0), (-1, -1), 'Korean')]))
                
                story.append(table)
                story.append(Spacer(1, 20))
            
            # 추천 이유
            if font_registered:
                story.append(Paragraph("추천 이유", styles['KoreanTitle']))
            else:
                story.append(Paragraph("Recommendation Reason", styles['Title']))
            story.append(Spacer(1, 10))
            
            recommendation_reason = ai_recommendation.get('추천이유', '')
            if recommendation_reason:
                # 긴 텍스트를 여러 문단으로 분할
                for line in recommendation_reason.split('\n'):
                    if line.strip():
                        if font_registered:
                            story.append(Paragraph(line.strip(), styles['Korean']))
                        else:
                            story.append(Paragraph(line.strip(), styles['Normal']))
                story.append(Spacer(1, 20))
        
        # 차트 이미지
        if chart_path and os.path.exists(chart_path):
            try:
                img = RLImage(chart_path, width=15*cm, height=8*cm)
                story.append(img)
            except Exception as e:
                print(f"[경고] 차트 이미지 삽입 실패: {e}")
        
        doc.build(story)
        print(f"[PDF 리포트 생성 완료]: {output_path}")
        return output_path
    
def generate_excel_report(ai_recommendation: dict, stock_analyses: dict, output_path: str = None) -> str:
        """
        Excel 리포트 생성
        
        Parameters:
            ai_recommendation: AI 추천 결과
            stock_analyses: 종목 분석 데이터
            output_path: 출력 파일 경로
        
        Returns:
            str: 생성된 파일 경로
        """
        if output_path is None:
            output_dir = "output"
            os.makedirs(output_dir, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = os.path.join(output_dir, f"stock_recommendation_report_{timestamp}.xlsx")
        
        # 추천 종목 데이터 준비
        recommended_stocks = ai_recommendation.get('추천종목', [])
        report_data = []
        
        for i, code in enumerate(recommended_stocks, 1):
            if code in stock_analyses:
                analysis = stock_analyses[code]
                tech_result = analysis.get('technical', {})
                stats = analysis.get('statistical', {})
                
                report_data.append({
                    '순위': i,
                    '종목명': analysis.get('stock_name', code),
                    '종목코드': code,
                    '이동평균신호': tech_result.get('이동평균_신호', 'N/A'),
                    'RSI신호': tech_result.get('RSI_신호', 'N/A'),
                    'RSI값': tech_result.get('RSI_값', 0),
                    'MACD신호': tech_result.get('MACD_신호', 'N/A'),
                    '기간수익률': stats.get('기간수익률', 0),
                    '샤프비율': stats.get('샤프비율', 0),
                    '변동성': stats.get('변동성', 0),
                    '일평균수익률': stats.get('일평균수익률', 0)
                })
        
        df_report = pd.DataFrame(report_data)
        
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            df_report.to_excel(writer, sheet_name='AI추천종목', index=False)
            
            # 추천 이유를 별도 시트에 저장
            if ai_recommendation.get('추천이유'):
                reason_df = pd.DataFrame({
                    '추천이유': [ai_recommendation['추천이유']]
                })
                reason_df.to_excel(writer, sheet_name='추천이유', index=False)
        
        print(f"[Excel 리포트 생성 완료]: {output_path}")
        return output_path
    
def send_email(sender_email: str, sender_password: str,
               recipient_email: str, subject: str, body: str,
               attachments: list = None) -> bool:
        """
        이메일 발송
        
        Parameters:
            sender_email: 발신자 이메일
            sender_password: 앱 비밀번호
            recipient_email: 수신자 이메일
            subject: 제목
            body: 본문
            attachments: 첨부파일 경로 리스트
        
        Returns:
            bool: 발송 성공 여부
        """
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject
        
        msg.attach(MIMEText(body, 'plain', 'utf-8'))
        
        if attachments:
            for path in attachments:
                if os.path.exists(path):
                    with open(path, 'rb') as f:
                        part = MIMEBase('application', 'octet-stream')
                        part.set_payload(f.read())
                        encoders.encode_base64(part)
                        part.add_header(
                            'Content-Disposition',
                            f'attachment; filename="{os.path.basename(path)}"'
                        )
                        msg.attach(part)
        
        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.send_message(msg)
            print(f"[이메일 발송 성공]: {recipient_email}")
            return True
        except Exception as e:
            print(f"[이메일 발송 실패]: {e}")
            return False

## 7. 메인 파이프라인

전체 프로세스를 통합하는 메인 함수입니다.
**참고: Module 01-04 전체 내용을 통합한 종합 프로젝트**

In [10]:
# ============================================
# 7. 메인 파이프라인
# **참고: Module 01-04 전체 내용을 통합한 종합 프로젝트**
# ============================================
def run_investment_analysis_pipeline(
    stock_codes: list,
    start_date: date = None,
    end_date: date = None,
    days: int = 90,
    output_dir: str = "output",
    send_email_flag: bool = False,
    sender_email: str = None,
    sender_password: str = None,
    recipient_email: str = None,
    ai_model_provider: str = "openai"  # "openai" 또는 "google_genai"
) -> dict:
    """
    투자 분석 자동화 파이프라인
    
    Parameters:
        stock_codes: 분석할 종목코드 리스트
        start_date: 시작일 (None이면 days 기준으로 계산)
        end_date: 종료일 (None이면 오늘)
        days: 분석 기간 (일)
        output_dir: 출력 디렉토리
        send_email_flag: 이메일 발송 여부
        sender_email: 발신자 이메일
        sender_password: 앱 비밀번호
        recipient_email: 수신자 이메일
        ai_model_provider: AI 모델 제공자 ("openai" 또는 "google_genai")
    
    Returns:
        dict: 분석 결과 (랭킹, 리포트 경로, AI 추천 등)
    """
    print("=" * 60)
    print("        투자 분석 자동화 시스템")
    print("=" * 60)
    
    # output 폴더 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # 날짜 설정
    if end_date is None:
        end_date = date.today()
    if start_date is None:
        start_date = end_date - timedelta(days=days)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 데이터 수집
    print("\n[1/6] 데이터 수집 중...")
    stock_data_dict = collect_multiple_stocks(stock_codes, start_date, end_date)
    
    if not stock_data_dict:
        print("[에러] 데이터 수집 실패")
        return {}
    
    print(f"  - 수집된 종목: {len(stock_data_dict)}개")
    
    # 2. 분석 수행
    print("\n[2/6] 분석 수행 중...")
    stock_analyses = {}
    
    for stock_code, df in stock_data_dict.items():
        stock_name = get_stock_name(stock_code)
        print(f"  - {stock_name} ({stock_code}) 분석 중...")
        
        # 수익률 계산
        df = calculate_returns(df)
        
        # 이동평균 추가
        df = add_moving_averages(df, periods=[5, 20, 60])
        
        # RSI 계산
        df = calculate_rsi(df)
        
        # MACD 계산
        df = calculate_macd(df)
        
        # 통계 계산
        stats = calculate_statistics(df)
        
        # 기술적 분석
        tech_result = technical_analysis(df)
        
        # 개별 종목 AI 분석 제거 (뉴스 크롤링도 제거)
        
        stock_analyses[stock_code] = {
            'stock_name': stock_name,
            'data': df,
            'technical': tech_result,
            'statistical': stats
        }
    
    # 3. AI 최적 종목 추천 (전체 종목 분석)
    print("\n[3/6] AI 최적 종목 추천 중...")
    ai_recommendation = recommend_optimal_stocks(stock_analyses, model_provider=ai_model_provider)
    
    print("\n[AI 추천 종목]")
    for i, code in enumerate(ai_recommendation['추천종목'], 1):
        stock_name = get_stock_name(code)
        print(f"  {i}. {stock_name} ({code})")
    print(f"\n[추천 이유]\n{ai_recommendation['추천이유']}")
    
    # 4. 시각화
    print("\n[4/6] 차트 생성 중...")
    recommendation_chart_path = os.path.join(output_dir, f"recommendation_chart_{timestamp}.png")
    create_recommendation_chart(ai_recommendation, stock_analyses, recommendation_chart_path)
    
    # 5. 리포트 생성
    print("\n[5/6] 리포트 생성 중...")
    
    pdf_path = generate_pdf_report(
        ai_recommendation,
        stock_analyses,
        chart_path=recommendation_chart_path,
        output_path=os.path.join(output_dir, f"stock_recommendation_report_{timestamp}.pdf")
    )
    
    excel_path = generate_excel_report(
        ai_recommendation,
        stock_analyses,
        output_path=os.path.join(output_dir, f"stock_recommendation_report_{timestamp}.xlsx")
    )
    
    # 6. 이메일 발송 (선택)
    if send_email_flag and sender_email and sender_password and recipient_email:
        print("\n[6/6] 이메일 발송 중...")
        
        # 이메일 본문 작성
        body = f"""AI 투자 종목 추천 리포트

생성 시각: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
분석 기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}

[AI 추천 종목]
"""
        for i, code in enumerate(ai_recommendation['추천종목'], 1):
            stock_name = get_stock_name(code)
            if code in stock_analyses:
                stats = stock_analyses[code].get('statistical', {})
                total_return = stats.get('기간수익률', 0)
                body += f"{i}. {stock_name} ({code}): 기간수익률 {total_return:.2f}%\n"
        
        body += f"\n[추천 이유]\n{ai_recommendation['추천이유'][:300]}...\n"
        body += "\n자세한 내용은 첨부된 리포트를 참고하세요."
        
        send_email(
            sender_email=sender_email,
            sender_password=sender_password,
            recipient_email=recipient_email,
            subject=f"AI 투자 종목 추천 리포트 ({end_date.strftime('%Y-%m-%d')})",
            body=body,
            attachments=[pdf_path, excel_path]
        )
    else:
        print("\n[6/6] 이메일 발송 건너뜀")
    
    # 완료
    print("\n" + "=" * 60)
    print("        파이프라인 완료!")
    print("=" * 60)
    print(f"\n결과물:")
    print(f"  - PDF: {pdf_path}")
    print(f"  - Excel: {excel_path}")
    print(f"  - Chart: {recommendation_chart_path}")
    
    return {
        'analyses': stock_analyses,
        'ai_recommendation': ai_recommendation,
        'pdf_path': pdf_path,
        'excel_path': excel_path,
        'chart_path': recommendation_chart_path
    }

## 8. 실행 예제

실제로 파이프라인을 실행하는 예제입니다.

In [11]:
# 분석할 종목 리스트
target_stocks = [
    "005930",  # 삼성전자
    "000660",  # SK하이닉스
    "035420",  # NAVER
    "051910",  # LG화학
    "006400",  # 삼성SDI
]

# 파이프라인 실행
result = run_investment_analysis_pipeline(
    stock_codes=target_stocks,
    days=90,  # 최근 90일 데이터
    output_dir="output",  # output 폴더에 저장
    send_email_flag=True,  # 이메일 발송은 .env 설정 후 True로 변경
    sender_email=os.getenv("GMAIL_ADDRESS"),
    sender_password=os.getenv("GMAIL_APP_PASSWORD"),
    recipient_email=os.getenv("RECIPIENT_EMAIL"),
    ai_model_provider="google_genai"  # openai 또는 google_genai
)

print("\n[AI 추천 결과]")
print(f"추천 종목: {result['ai_recommendation']['추천종목']}")
print(f"추천 이유:\n{result['ai_recommendation']['추천이유']}")

        투자 분석 자동화 시스템

[1/6] 데이터 수집 중...
  - 수집된 종목: 5개

[2/6] 분석 수행 중...
  - 삼성전자 (005930) 분석 중...
  - SK하이닉스 (000660) 분석 중...
  - NAVER (035420) 분석 중...
  - LG화학 (051910) 분석 중...
  - 삼성SDI (006400) 분석 중...

[3/6] AI 최적 종목 추천 중...

[AI 추천 종목]
  1. 삼성SDI (006400)
  2. 삼성전자 (005930)

[추천 이유]

애널리스트로서 제공된 원시 데이터를 면밀히 분석한 결과, 현재 시장 상황과 투자 목표를 고려할 때 다음과 같은 종목들을 추천합니다. 특히, 과매수/과매도 신호와 과거 수익성 및 위험 대비 수익률을 종합적으로 고려하여 실용적인 관점에서 접근했습니다.

[4/6] 차트 생성 중...

[5/6] 리포트 생성 중...
[PDF 리포트 생성 완료]: output\stock_recommendation_report_20260118_092950.pdf
[Excel 리포트 생성 완료]: output\stock_recommendation_report_20260118_092950.xlsx

[6/6] 이메일 발송 중...
[이메일 발송 성공]: youngjea.oh@oyj.ai.kr

        파이프라인 완료!

결과물:
  - PDF: output\stock_recommendation_report_20260118_092950.pdf
  - Excel: output\stock_recommendation_report_20260118_092950.xlsx
  - Chart: output\recommendation_chart_20260118_092950.png

[AI 추천 결과]
추천 종목: ['006400', '005930']
추천 이유:

애널리스트로서 제공된 원시 데이터를 면밀히 분석한 결과, 현재 시장 상황과 투자 목표를 고려할 때 다음과 같은 종목들을 